# Assignment 3 Question 1

### CO 487/687 Applied Cryptography Fall 2023 

This Jupyter notebook contains Python 3 code for Assignment 3 Question 1 on "Symemtric Encryption in Python".

### Documentation

- [Python cryptography library](https://cryptography.io/en/latest/)

The following code imports all the required functions for the assignment.

In [1]:
import base64
import getpass
import os
import sys
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.hmac import HMAC
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes    
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives import constant_time

HKMAC_NITERS = 200000
# Params of AES128
AES_KEYSIZE = BLOCKSIZE = HMAC_KEYSIZE = 16
BLOCKSIZE_BITS = 128
# Params of SHA3-256
TAGSIZE = 32

These two functions convert a byte array into a printable string and back, which might be helpful to you since cryptographic routines often work with byte arrays.

In [2]:
def bytes2string(b):
    return base64.urlsafe_b64encode(b).decode('utf-8')

def string2bytes(s):
    return base64.urlsafe_b64decode(s.encode('utf-8'))

Implement the main encryption function below. Your function will take as input a string, and will output a string or dictionary containing all the values needed to decrypt (other than the password, of course). The code below will prompt the user to enter their password during encryption.

In [3]:
def derive_cipher_suite(password: bytes, nonce: bytes | None = None):
    """Return the padder, the cipher, and the MAC from a single password

    If an input nonce is given, it will be used for the Block cipher CTR mode;
    otherwise a random nonce will be generated
    """
    hkdf = PBKDF2HMAC(
        algorithm=hashes.SHA3_256(),
        length=AES_KEYSIZE + HMAC_KEYSIZE,  # we need two keys
        iterations=HKMAC_NITERS,
        salt=b"",  # TODO: need a place to store salt
    )
    key = hkdf.derive(password)
    key_sign = key[:HMAC_KEYSIZE]
    key_enc = key[HMAC_KEYSIZE:]
    pad = padding.PKCS7(BLOCKSIZE_BITS)
    nonce = os.urandom(BLOCKSIZE) if nonce is None else nonce
    cipher = Cipher(algorithms.AES128(key_enc), modes.CTR(nonce))
    mac = HMAC(key=key_sign, algorithm=hashes.SHA3_256())

    return pad, cipher, nonce, mac

def encrypt(message):
    
    # encode the string as a byte string, since cryptographic functions usually work on bytes
    plaintext = message.encode('utf-8')

    # Use getpass to prompt the user for a password
    password = getpass.getpass("Enter password:")
    password2 = getpass.getpass("Enter password again:")

    # Do a quick check to make sure that the password is the same!
    if password != password2:
        sys.stderr.write("Passwords did not match")
        sys.exit()

    ### START: This is what you have to change
    
    pad, cipher, nonce, mac = derive_cipher_suite(password.encode())
    encryptor = cipher.encryptor()
    padder = pad.padder()

    plaintext = padder.update(plaintext) + padder.finalize()
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()
    mac.update(ciphertext)
    tag = mac.finalize()

    return bytes2string(nonce + ciphertext + tag)
    
    ### END: This is what you have to change

Now we call the `encrypt` function with a message, and print out the ciphertext it generates.

In [4]:
mymessage = "Hello, world!"
ciphertext = encrypt(mymessage)
print(ciphertext)

e3peon4UIfMayWmhmq3c881oJ1EcaAg3NcB2zXEIsU_zK3pKLj2b9989E74yPfnIHgS8MwKhjPLE5sOiXkSB-w==


Implement the main decryption function below.  Your function will take as input the string or dictionary output by `encrypt`, prompt the user to enter the password, and then do all the relevant cryptographic operations.

In [5]:
def decrypt(ciphertext):
    # prompt the user for the password
    password = getpass.getpass("Enter the password:")

    ### START: This is what you have to change
    ciphertext = string2bytes(ciphertext)
    nonce = ciphertext[:BLOCKSIZE]
    tag = ciphertext[-TAGSIZE:]
    ciphertext = ciphertext[BLOCKSIZE:-TAGSIZE]
    pad, cipher, nonce, mac = derive_cipher_suite(password.encode(), nonce)
    decryptor = cipher.decryptor()
    unpadder = pad.unpadder()

    mac.update(ciphertext)
    mac.verify(tag)

    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    plaintext = unpadder.update(plaintext) + unpadder.finalize()
    
    ### END: This is what you have to change

    # decode the byte string back to a string
    return plaintext.decode('utf-8')

Now let's try decrypting the ciphertext you encrypted above by entering the same password as you used for encryption.

In [6]:
mymessagedecrypted = decrypt(ciphertext)
print(mymessagedecrypted)
assert mymessagedecrypted == mymessage

Hello, world!


Try again but this time see what happens if you use a different password to decrypt. Your function should fail.

In [7]:
mymessagedecrypted = decrypt(ciphertext)
print(mymessagedecrypted)
assert mymessagedecrypted == mymessage

InvalidSignature: Signature did not match digest.